In [ ]:
from train_utils import *

In [ ]:
train_model(DATASET='potsdam', MODEL_TYPE='FCN', BACKBONE='r101', NUM_EPOCHS=10, LEARNING_RATE=0.01, BATCH_SIZE = 2)
#train_model(DATASET='loveda', MODEL_TYPE='FCN', BACKBONE='r101', NUM_EPOCHS=10, LEARNING_RATE=0.01, BATCH_SIZE = 2)

### Evaluate the Model 

In [ ]:
# Load the model
model = torch.load(config['data']['model_weights']+'/potsdam_FCN.pth.tar')
optimizer = torch.load(config['data']['model_weights']+'/potsdam_FCN_optim.pth.tar')

In [ ]:
# Get all images, targets and predictions for the test set
img_size = 512
all_images = np.zeros([BATCH_SIZE*(len(test_loader)-1), 3, img_size, img_size])
all_labels = np.zeros([BATCH_SIZE*(len(test_loader)-1), img_size, img_size])
all_predictions = np.zeros([BATCH_SIZE*(len(test_loader)-1), img_size, img_size])

for idx, (data, label) in enumerate(test_loader):
    print(f"Processing batch {idx} of {len(test_loader)-1}")
    if idx != len(test_loader)-1:
        all_images[idx*BATCH_SIZE:((idx+1)*BATCH_SIZE), :, :, :] = data
        all_labels[idx*BATCH_SIZE:((idx+1)*BATCH_SIZE), :, :] = label

        # Get model predictions and save them
        model.eval()
        with torch.no_grad():
            data = data.to(DEVICE)
            logits = model(data)['out']
            prob = nn.Softmax(logits).dim
            pred = torch.argmax(prob, dim = 1)
        all_predictions[idx*BATCH_SIZE:((idx+1)*BATCH_SIZE), :, :] = pred.cpu()

In [ ]:
def calculate_label_area(label):
    total_pixel = label.size
    rel = pd.Series([100*label[label==i].size/total_pixel for i in range(NUM_CLASSES)])
    rel.index = Label_classes
    return rel
def plot_areas(label, prediction):
    label_rel = calculate_label_area(label)
    pred_rel = calculate_label_area(prediction)
    df = pd.DataFrame([label_rel, pred_rel], columns=Label_classes, index = ['target', 'prediction']).transpose().reset_index().melt(id_vars='index', var_name='type')
    
    plt.style.use("dark_background")
    fig, ax = plt.subplots()
    sns.barplot(x='index', y='value', hue='type', data=df)
    plt.xlabel("Class")
    plt.ylabel("Relative Surface")
    plt.title("Area of classes in target(left) and prediction(right)")
    plt.xticks(rotation = 90)

    # Change coloring to same colors as in label mask
    class_colors_hex = {k: '#%02x%02x%02x' % v for k, v in idx2rgb.items()}
    col_list= list(class_colors_hex.values())
    col_list = col_list+col_list
    for i, bar in enumerate(ax.patches):
        col = col_list[i]
        bar.set_color(col)
    for container in ax.containers:
        ax.bar_label(container, fmt='%.1f')
    ax.get_legend().remove()
    plt.show()


In [ ]:
# Plot random instance of the test set
idx = random.randint(0, all_images.shape[0]-1)
img = all_images[idx,:,:,:]
label = all_labels[idx,:,:]
prediction = all_predictions[idx, :, :]

fig, ax = plt.subplots(1,4, figsize = (15,4))
ax[0].imshow(ToPILImage()(torch.from_numpy(img)))
ax[0].set_title('Noramlized Input Image')
ax[1].imshow(ToPILImage()(torch.from_numpy(denormalize_image(img))))
ax[1].set_title('Denormalized Input Image')
ax[2].imshow(map_label2rgb(label))
ax[2].set_title('Target')
ax[3].imshow(map_label2rgb(prediction))
ax[3].set_title('Prediction')
plt.suptitle('Random Image and Prediction from Test Set')
plt.show()

In [ ]:
plot_areas(label, prediction)